Project 22

In [ ]:
# data load


In [ ]:
# simple CNN model

# import libraries
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

# create model as a class using pytorch
# 11 channel input, one channel output
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 11 input image channel, 6 output channels, 5x5 square convolution
        self.conv1 = nn.Conv2d(11, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5x5 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 1)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

# create model
net = Net()
print(net)
